In [1]:
from __future__ import print_function
from __future__ import division

from sklearn.datasets import make_classification
from sklearn.cross_validation import cross_val_score
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.svm import SVC

from bayes_opt import BayesianOptimization

# Load data set and target values
data, target = make_classification(n_samples=2500,
                                   n_features=45,
                                   n_informative=12,
                                   n_redundant=7)

def svccv(C, gamma):
    return cross_val_score(SVC(C=C, gamma=gamma, random_state=2),
                           data, target, 'f1', cv=5).mean()

def rfccv(n_estimators, min_samples_split, max_features):
    return cross_val_score(RFC(n_estimators=int(n_estimators),
                               min_samples_split=int(min_samples_split),
                               max_features=min(max_features, 0.999),
                               random_state=2),
                           data, target, 'f1', cv=5).mean()

In [2]:
svcBO = BayesianOptimization(svccv, {'C': (0.001, 100), 'gamma': (0.0001, 0.1)})
svcBO.explore({'C': [0.001, 0.01, 0.1], 'gamma': [0.001, 0.01, 0.1]})

rfcBO = BayesianOptimization(rfccv, {'n_estimators': (10, 250),
                                     'min_samples_split': (2, 25),
                                     'max_features': (0.1, 0.999)})
svcBO.maximize()

print('-'*53)
rfcBO.maximize()

print('-'*53)
print('Final Results')
print('SVC: %f' % svcBO.res['max']['max_val'])
print('RFC: %f' % rfcBO.res['max']['max_val'])

Initialization
-----------------------------------------------------
 Step |   Time |      Value |         C |     gamma | 
    1 | 00m02s |    0.66702 |    0.0010 |    0.0010 | 
    2 | 00m02s |    0.75646 |    0.0100 |    0.0100 | 
    3 | 00m02s |    0.66702 |    0.1000 |    0.1000 | 
    4 | 00m02s |    0.84213 |   72.8943 |    0.0852 | 
    5 | 00m02s |    0.86229 |   19.4868 |    0.0774 | 
    6 | 00m02s |    0.92655 |   36.5293 |    0.0425 | 
    7 | 00m02s |    0.83926 |   91.7076 |    0.0861 | 
    8 | 00m02s |    0.86759 |   99.1940 |    0.0757 | 
Bayesian Optimization
-----------------------------------------------------
 Step |   Time |      Value |         C |     gamma | 
    9 | 00m08s |    0.90700 |   38.9650 |    0.0508 | 
   10 | 00m04s |    0.92655 |   12.9470 |    0.0425 | 
   11 | 00m04s |    0.94152 |   23.0225 |    0.0350 | 
   12 | 00m04s |    0.95323 |   48.2991 |    0.0274 | 
   13 | 00m05s |    0.94390 |   71.0984 |    0.0323 | 
   14 | 00m04s |    0.96084 | 